In [24]:
import pandas as pd
from snsynth import Synthesizer
import matplotlib.pyplot as plt
import seaborn as sns
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pandas as pd
import plotly.graph_objects as go
from snsynth.mst import MSTSynthesizer




In [25]:
#aldult.data
pums = pd.read_csv("adult.data", header=None, names=[
    "age", "workclass", "fnlwgt", "education", "education_num", "marital_status", "occupation", 
    "relationship", "race", "sex", "capital_gain", "capital_loss", "hours_per_week", "native_country", "income"])

print(pums.head())  

   age          workclass  fnlwgt   education  education_num  \
0   39          State-gov   77516   Bachelors             13   
1   50   Self-emp-not-inc   83311   Bachelors             13   
2   38            Private  215646     HS-grad              9   
3   53            Private  234721        11th              7   
4   28            Private  338409   Bachelors             13   

        marital_status          occupation    relationship    race      sex  \
0        Never-married        Adm-clerical   Not-in-family   White     Male   
1   Married-civ-spouse     Exec-managerial         Husband   White     Male   
2             Divorced   Handlers-cleaners   Not-in-family   White     Male   
3   Married-civ-spouse   Handlers-cleaners         Husband   Black     Male   
4   Married-civ-spouse      Prof-specialty            Wife   Black   Female   

   capital_gain  capital_loss  hours_per_week  native_country  income  
0          2174             0              40   United-States   <=50

In [58]:


# epsilon = 1.0
synth = Synthesizer.create("mst", epsilon=3.0, verbose=True)
synth.fit(pums, preprocessor_eps=1.0)
pums_synth = synth.sample(1000)





Spent 1.0 epsilon on preprocessor, leaving 2.0 for training
Fitting with 8366677401600000 dimensions
Getting cliques
Estimating marginals


In [62]:
# epsilon = 100.0

synth_100 = Synthesizer.create("mst", epsilon=102.0, verbose=True)
synth_100.fit(pums, preprocessor_eps=100.0)
pums_synth_1000 = synth_100.sample(1000)

Spent 100.0 epsilon on preprocessor, leaving 2.0 for training
Fitting with 8366677401600000 dimensions
Getting cliques
Estimating marginals


In [63]:
#print(pums_synth.head())

print(pums_synth_1000.head())


   age   workclass    fnlwgt      education  education_num  \
0   40     Private  321536.0      Bachelors             13   
1   24     Private  112640.0    Prof-school             15   
2   27   State-gov  112640.0        Masters             14   
3   30     Private  112640.0   Some-college             10   
4   37     Private  321536.0      Bachelors             13   

        marital_status        occupation    relationship    race   sex  \
0             Divorced    Prof-specialty   Not-in-family   White  male   
1        Never-married   Exec-managerial   Not-in-family   White  male   
2              Widowed     Other-service       Unmarried   White  male   
3   Married-civ-spouse             Sales         Husband   White  male   
4   Married-civ-spouse             Sales         Husband   White  male   

   capital_gain  capital_loss  hours_per_week  native_country  income  
0        6553.6         409.6              43   United-States   <=50K  
1        6553.6         409.6         

In [28]:
print(pums.head())


   age          workclass  fnlwgt   education  education_num  \
0   39          State-gov   77516   Bachelors             13   
1   50   Self-emp-not-inc   83311   Bachelors             13   
2   38            Private  215646     HS-grad              9   
3   53            Private  234721        11th              7   
4   28            Private  338409   Bachelors             13   

        marital_status          occupation    relationship    race      sex  \
0        Never-married        Adm-clerical   Not-in-family   White     Male   
1   Married-civ-spouse     Exec-managerial         Husband   White     Male   
2             Divorced   Handlers-cleaners   Not-in-family   White     Male   
3   Married-civ-spouse   Handlers-cleaners         Husband   Black     Male   
4   Married-civ-spouse      Prof-specialty            Wife   Black   Female   

   capital_gain  capital_loss  hours_per_week  native_country  income  
0          2174             0              40   United-States   <=50

In [30]:
# Afficher les valeurs uniques de la colonne 'sex'
print(pums['sex'].unique())


[' Male' ' Female']


In [64]:
# Suppression des espaces et mise en minuscules 
pums['sex'] = pums['sex'].str.strip().str.lower()
pums_synth['sex'] = pums_synth['sex'].str.strip().str.lower()
pums_synth_1000['sex'] = pums_synth_1000['sex'].str.strip().str.lower()


female_data = pums[pums['sex'] == 'female']
female_data_synth = pums_synth[pums_synth['sex'] == 'female']
female_data_synth_1000 = pums_synth_1000[pums_synth_1000['sex'] == 'female']

male_data = pums[pums['sex'] == 'male']
male_data_synth = pums_synth[pums_synth['sex'] == 'male']
male_data_synth_1000 = pums_synth_1000[pums_synth_1000['sex'] == 'male']

#print(female_data.head())
print(female_data_synth.head())


    age workclass    fnlwgt      education  education_num  \
1    54   Private  171212.8        HS-grad              9   
7    21   Private   67993.6   Some-college             10   
9    43   Private   67993.6           11th              7   
11   33   Private  171212.8        HS-grad              9   
12   31   Private  377651.2           10th             14   

         marital_status          occupation relationship    race     sex  \
1    Married-civ-spouse               Sales         Wife   Black  female   
7         Never-married        Adm-clerical    Own-child   Other  female   
9         Never-married   Machine-op-inspct    Own-child   White  female   
11        Never-married        Adm-clerical    Unmarried   White  female   
12        Never-married        Craft-repair    Own-child   White  female   

    capital_gain  capital_loss  hours_per_week  native_country  income  
1         6553.6         204.8              40   United-States    >50K  
7         6553.6         204.8

In [65]:
# La retraite à 60 ans
female_data = female_data[female_data['age'] <= 60]
female_data_synth = female_data_synth[female_data_synth['age'] <= 60]
female_data_synth_1000 = female_data_synth_1000[female_data_synth_1000['age'] <= 60]


male_data = male_data[male_data['age'] <= 60]
male_data_synth = male_data_synth[male_data_synth['age'] <= 60]
male_data_synth_1000 = male_data_synth_1000[male_data_synth_1000['age'] <= 60]



# Longueur des données pour les hommes et les femmes
print(len(female_data))
print(len(male_data))

10029
20200


In [66]:


# Calculer la moyenne des heures par semaine pour les femmes par tranche d'âge
women_avg_hours = female_data.groupby('age')['hours_per_week'].mean().reset_index()
women_avg_hours_synth = female_data_synth.groupby('age')['hours_per_week'].mean().reset_index()
women_avg_hours_synth_1000 = female_data_synth_1000.groupby('age')['hours_per_week'].mean().reset_index()

# Calculer la moyenne des revenus pour les hommes par tranche d'âge
men_avg_income = male_data.groupby('age')['hours_per_week'].mean().reset_index()
men_avg_income_synth = male_data_synth.groupby('age')['hours_per_week'].mean().reset_index()
men_avg_income_synth_1000 = male_data_synth_1000.groupby('age')['hours_per_week'].mean().reset_index()

#print(female_data.head())
#print(female_data_synth.head())

print(women_avg_hours.mean())
print('-------------------')
print(women_avg_hours_synth.mean())
print('----------------')
print(women_avg_hours_synth_1000.mean())


age               38.500000
hours_per_week    37.302894
dtype: float64
-------------------
age               38.023256
hours_per_week    41.094913
dtype: float64
----------------
age               38.500000
hours_per_week    38.430476
dtype: float64


In [68]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Créer un subplot avec 2 lignes
fig = make_subplots(rows=2, cols=1, 
                    subplot_titles=("Heures par semaine des Femmes", "Revenu des Hommes"))

# Ajout des heures par semaine pour les femmes (rose)
fig.add_trace(go.Scatter(
    x=women_avg_hours['age'], 
    y=women_avg_hours['hours_per_week'], 
    mode='lines+markers',
    name='Heures par semaine (Femmes)',
    line=dict(color='lightpink'),  # Rose clair
    marker=dict(color='pink'),       # Rose
), row=1, col=1)

# Ajout des heures par semaine pour les femmes dans synth (rose)
fig.add_trace(go.Scatter(
    x=women_avg_hours_synth['age'], 
    y=women_avg_hours_synth['hours_per_week'], 
    mode='lines+markers',
    name='Heures par semaine (Femmes Synth)',
    line=dict(color='red'),          # Rose
    marker=dict(color='lightpink')   # Rose clair
), row=1, col=1)

# Ajout des heures par semaine pour les femmes dans synth 1000 (rose)

fig.add_trace(go.Scatter(
    x = women_avg_hours_synth_1000['age'],
    y = women_avg_hours_synth_1000['hours_per_week'],
    mode = 'lines+markers',
    name = 'Heures par semaine (Femmes Synth 100)',
    line = dict(color = 'orange'),
    marker = dict(color = 'orange')
), row = 1, col = 1)

# Ajout des revenus pour les hommes (bleu)
fig.add_trace(go.Scatter(
    x=men_avg_income['age'], 
    y=men_avg_income['hours_per_week'], 
    mode='lines+markers',
    name='Heures par semaines (Hommes)',
    line=dict(color='blue'),         # Bleu
    marker=dict(color='blue')   # Bleu clair
), row=2, col=1)

# Ajout des revenus pour les hommes dans synth (bleu clair)
fig.add_trace(go.Scatter(
    x=men_avg_income_synth['age'], 
    y=men_avg_income_synth['hours_per_week'], 
    mode='lines+markers',
    name='Heures par semaines (Hommes Synth)',
    line=dict(color='lightblue'),    # Bleu clair
    marker=dict(color='lightblue')         # Bleu
), row=2, col=1)

# Ajout des revenus pour les hommes dans synth 1000 (bleu clair)
fig.add_trace(go.Scatter(
    x=men_avg_income_synth_1000['age'],
    y=men_avg_income_synth_1000['hours_per_week'],
    mode='lines+markers',
    name='Heures par semaines (Hommes Synth 100)',
    line=dict(color='green'),
    marker=dict(color='green')
), row=2, col=1)


# Mise en forme du graphique
fig.update_layout(
    title="Comparaison des heures par semaine des Femmes et des Hommes selon l'Âge",
    xaxis_title="Âge",
    yaxis_title="Moyenne des heures par semaine",
    legend_title="Variable",
    template="plotly_white"
)

# Affichage du graphique
fig.show()
